In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import os

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.law.go.kr/LSW/lsAstSc.do?menuId=391&subMenuId=397&tabMenuId=437&query=#AJAX')
time.sleep(3)

In [3]:
driver.find_element(By.ID, 'liLsFd').find_element(By.XPATH, 'parent::a').click()
time.sleep(2)
driver.find_element(By.ID, 'divLsFd').find_element(By.ID, 'lsFd09').click()
time.sleep(2)
driver.find_element(By.ID, 'efCheck').click()
time.sleep(2)
driver.execute_script('setEfChk(); return;')

In [5]:
def get_row_per_page():
    main_window = driver.current_window_handle

    전체법령 = driver.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
    for 법령 in 전체법령:
        row = 법령.find_element(By.CLASS_NAME, 'ctn1').find_element(By.TAG_NAME, 'a')
        header = row.text
        row.click()
        time.sleep(3)

        for window in driver.window_handles:
            if window != main_window:
                driver.switch_to.window(window)
                break

        page_source = driver.find_element(By.ID, 'contentBody').get_attribute('innerHTML')
        time.sleep(1)
        match = re.search(
            r'<!--\s*조문정보\s*-->[\s\S]*?<!--\s*조문정보\s*-->',
            page_source,
            re.DOTALL
        )

        soup = BeautifulSoup(match.group(0), 'html.parser')
        content = soup.get_text(strip=True, separator='')

        os.makedirs(f'./datas/texts', exist_ok=True)
        with open(f'./datas/texts/{header}.txt', encoding='utf-8', mode='w') as f:
            f.write(content)

        driver.close()
        time.sleep(1)
        driver.switch_to.window(main_window)

In [ ]:
count = driver.find_element(By.CLASS_NAME, 'paging').find_elements(By.TAG_NAME, 'li')

for i in range(2, len(count)-1):
    paging = driver.find_element(By.CLASS_NAME, 'paging').find_elements(By.TAG_NAME, 'li')
    paging[i].click()
    time.sleep(3)
    get_row_per_page()